In [151]:
import numpy as np
from scipy.stats import norm
import math

In [107]:
import numpy as np

class FDoptionPricer:
    allowed_methods = {'explicit', 'implicit', 'cn'}
    allowed_styles = {'european'}
    allowed_types = {'put', 'call'}
    
    def __init__(self, optionStyle, optionType, S0, K, T = 1, r = 0.07, sigma = 0.25):
        if optionStyle not in self.allowed_styles or optionType not in self.allowed_types:
            raise ValueError('style/type not allowed')
        self.style = optionStyle
        self.type = optionType
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma

    def getPrice(self, method, M = 1000, N = 100):
        if method.lower() not in self.allowed_methods:
            raise ValueError('not an allowed method')

        x = np.linspace(-3*self.sigma*np.sqrt(self.T), 3*self.sigma*np.sqrt(self.T), N)
        q = 2 * self.r / self.sigma**2
        dt = (self.sigma**2 * self.T / 2) / M
        dx = x[1] - x[0]
        lam = dt / dx**2

        u = np.maximum(np.exp((q + 1)/2 * x) - np.exp((q - 1)/2 * x), 0)

        def gta(a, b, c, d):
            n = len(d)
            cp = np.zeros(n)
            dp = np.zeros(n)
            cp[0] = c[0] / b[0]
            dp[0] = d[0] / b[0]
            for i in range(1, n):
                denom = b[i] - a[i-1] * cp[i-1]
                cp[i] = c[i] / denom if i < n-1 else 0
                dp[i] = (d[i] - a[i-1] * dp[i-1]) / denom
            x = np.zeros(n)
            x[-1] = dp[-1]
            for i in reversed(range(n-1)):
                x[i] = dp[i] - cp[i] * x[i+1]
            return x

        if method == 'explicit':
            for m in range(1, M):
                w = np.zeros(N)
                w[1:-1] = lam * u[:-2] + (1 - 2*lam) * u[1:-1] + lam * u[2:]
                w[0] = 0
                t = (m-1) * dt
                w[-1] = np.exp((q + 1)/2 * x[-1] + ((q + 1)**2 / 4) * t) - np.exp((q - 1)/2 * x[-1] + ((q - 1)**2 / 4) * t)
                u = w
            V = u * self.K * np.exp(-(q - 1)/2 * x - ((q + 1)**2 / 4) * self.sigma**2 * self.T / 2)
            S = self.K * np.exp(x)
            logS0 = np.log(self.S0 / self.K)
            i = np.searchsorted(x, logS0) - 1
            i = np.clip(i, 0, N - 2)
            w = (logS0 - x[i]) / dx
            price = V[i] * (1 - w) + V[i+1] * w

        if method == 'implicit':
            a = -lam * np.ones(N-2)
            b = (1 + 2*lam) * np.ones(N-2)
            c = -lam * np.ones(N-2)
            for m in range(1, M):
                t = (m-1) * dt
                rhs = u[1:-1].copy()
                rhs[0] += lam * 0
                u_right = np.exp((q + 1)/2 * x[-1] + ((q + 1)**2 / 4) * t) - np.exp((q - 1)/2 * x[-1] + ((q - 1)**2 / 4) * t)
                rhs[-1] += lam * u_right
                u_inner = gta(a, b, c, rhs)
                u = np.concatenate(([0], u_inner, [u_right]))
            V = u * self.K * np.exp(-(q - 1)/2 * x - ((q + 1)**2 / 4) * self.sigma**2 * self.T / 2)
            logS0 = np.log(self.S0 / self.K)
            i = np.searchsorted(x, logS0) - 1
            i = np.clip(i, 0, N - 2)
            w = (logS0 - x[i]) / dx
            price = V[i] * (1 - w) + V[i+1] * w

        if method == 'cn':
            aL = -0.5 * lam * np.ones(N-2)
            bL = (1.0 + lam) * np.ones(N-2)
            cL = -0.5 * lam * np.ones(N-2)
            for m in range(1, M):
                t_m  = (m-1) * dt
                t_m1 = m * dt
                uR_m  = np.exp((q + 1)/2 * x[-1] + ((q + 1)**2 / 4) * t_m) - np.exp((q - 1)/2 * x[-1] + ((q - 1)**2 / 4) * t_m)
                uR_m1 = np.exp((q + 1)/2 * x[-1] + ((q + 1)**2 / 4) * t_m1) - np.exp((q - 1)/2 * x[-1] + ((q - 1)**2 / 4) * t_m1)
                u_old = u
                rhs = (0.5*lam) * u_old[0:N-2] + (1.0 - lam) * u_old[1:N-1] + (0.5*lam) * u_old[2:N]
                rhs[-1] += 0.5 * lam * (uR_m + uR_m1)
                u_inner = gta(aL, bL, cL, rhs)
                u = np.concatenate(([0.0], u_inner, [uR_m1]))
            V = u * self.K * np.exp(-(q - 1)/2 * x - ((q + 1)**2 / 4) * self.sigma**2 * self.T / 2)
            logS0 = np.log(self.S0 / self.K)
            i = np.searchsorted(x, logS0) - 1
            i = np.clip(i, 0, N - 2)
            w_lin = (logS0 - x[i]) / dx
            price = V[i] * (1 - w_lin) + V[i+1] * w_lin

        return price

In [127]:
eurocall = FDoptionPricer('european', 'call', 100, 100)
eurocall.getPrice(method = 'explicit', N = 100, M = 100)

1.0053684604637031e+60

In [131]:
T = 1 #period of contract
S_0 = 100  #price at time zero
K = 100  #exercise price
sigma = 0.25  #Volatility
r = 0.07  #Risk-neutral interest-rate
price = 0  #Just initialization :)
S_max = 1000

N = 500
M = 50
dt = T / N
ds = S_max / M

f = np.zeros((M+1,N+1))  # The array f is the mesh of approximation of the option price function
I = np.arange(0, M+1)
J = np.arange(0, N+1)

# Boundary and final conditions
f[:, N] = np.maximum(K - (I * ds), 0)
f[0, :] = K * np.exp(-r * (T - J * dt))
f[M, :] = 0

alpha = 0.25 * dt * (sigma**2 * (I**2) - r * I)
beta = -dt * 0.5 * (sigma**2 * (I**2) + r)
gamma = 0.25 * dt * (sigma**2 * (I**2) + r * I)

M1 = np.diag(1-beta[1:M]) + np.diag(-alpha[2:M], k=-1) + np.diag(-gamma[1:M-1], k=1)
M2 = np.diag(1+beta[1:M]) + np.diag(alpha[2:M], k=-1) + np.diag(gamma[1:M-1], k=1)

for j in range(N-1, -1, -1):
    l = np.zeros(M - 1)
    l[0] = alpha[1] * (f[0, j] + f[0, j+1])
    l[-1] = gamma[M-1] * (f[M, j] + f[M, j+1])
    f[1:M, j] = np.linalg.solve(M1, M2 @ f[1:M, j+1] + l)

## Finding the price by interapolation
idown = int(np.floor(S_0 / ds))
iup = int(np.ceil(S_0 / ds))
print(idown)
print(iup)
if idown == iup:
    price = f[idown, 0]
else:
    price = f[idown, 0] + ((iup - (S_0 / ds)) / (iup - idown)) * (f[iup, 0] - f[idown, 0])

print(price)

5
5
5.675600670643971


In [29]:
def thomas(a, b, c, d):
    n = b.size
    cp = np.empty(n-1); dp = np.empty(n)
    cp[0] = c[0]/b[0]; dp[0] = d[0]/b[0]
    for i in range(1, n-1):
        den = b[i] - a[i-1]*cp[i-1]
        cp[i] = c[i]/den
        dp[i] = (d[i] - a[i-1]*dp[i-1])/den
    dp[-1] = (d[-1] - a[-1]*dp[-2])/(b[-1] - a[-1]*cp[-1])
    x = np.empty(n); x[-1] = dp[-1]
    for i in range(n-2, -1, -1):
        x[i] = dp[i] - cp[i]*x[i+1]
    return x

def crank_nicolson_european_call(S0, K, T, r, sigma, N_s=200, N_t=400, S_max=None):
    if S_max is None:
        S_max = max(2.0*S0, 4.0*K)
    dt = T / N_t
    S = np.linspace(0.0, S_max, N_s+1)
    V = np.maximum(S - K, 0.0)

    i = np.arange(1, N_s)
    alpha = 0.25*dt*(sigma**2*i**2 - r*i)
    beta  = -0.5*dt*(sigma**2*i**2 + r)
    gamma = 0.25*dt*(sigma**2*i**2 + r*i)

    aL = -alpha[1:]
    bL = 1.0 - beta
    cL = -gamma[:-1]

    from scipy.sparse import diags
    M_R = diags([alpha[1:], (1.0 + beta), gamma[:-1]], [-1, 0, 1], shape=(N_s-1, N_s-1)).tocsc()

    for n in range(N_t):
        t   = n*dt
        t1  = (n+1)*dt
        V0n = 0.0
        V0p = 0.0
        Vsn = S_max - K*np.exp(-r*(T - t))
        Vsp = S_max - K*np.exp(-r*(T - t1))

        rhs = M_R.dot(V[1:-1])
        rhs[0]  += alpha[0]*V0n + alpha[0]*V0p
        rhs[-1] += gamma[-1]*Vsn + gamma[-1]*Vsp

        V[1:-1] = thomas(aL, bL, cL, rhs)
        V[0]  = V0p
        V[-1] = Vsp

    return float(np.interp(S0, S, V))

In [125]:
price = crank_nicolson_european_call(100, 100, 1.0, 0.07, 0.25, N_s=100, N_t=400)
price

13.332450229933379

In [43]:
def implicit_european_call(S0, K, T, r, sigma, N_s=200, N_t=400, S_max=None):
    if S_max is None:
        S_max = max(2.0*S0, 4.0*K)
    dt = T / N_t
    S = np.linspace(0.0, S_max, N_s+1)
    V = np.maximum(S - K, 0.0)

    i = np.arange(1, N_s)
    alpha = 0.5*dt*(sigma**2*i**2 - r*i)
    beta  = -dt*(sigma**2*i**2 + r)
    gamma = 0.5*dt*(sigma**2*i**2 + r*i)

    a = -alpha[1:]
    b = 1.0 - beta
    c = -gamma[:-1]

    for n in range(N_t):
        t1 = (n+1)*dt
        V0 = 0.0
        VS = S_max - K*np.exp(-r*(T - t1))
        rhs = V[1:-1].copy()
        rhs[0]  += alpha[0]*V0
        rhs[-1] += gamma[-1]*VS
        V[1:-1] = thomas(a, b, c, rhs)
        V[0]  = V0
        V[-1] = VS

    return float(np.interp(S0, S, V))

In [123]:
price = implicit_european_call(100, 100, 1.0, 0.07, 0.25, N_s=100, N_t=400)
price

13.329004213602238

In [155]:
def explicit_european_call(S0, K, T, r, sigma, N_s=200, N_t=400, S_max=None):
    if S_max is None:
        S_max = max(2.0*S0, 4.0*K)

    S = np.linspace(0.0, S_max, N_s+1, dtype=float)
    V = np.maximum(S - K, 0.0)

    i = np.arange(1, N_s, dtype=float)
    dt = T / float(N_t)

    alpha = 0.5*dt*(sigma**2 * i**2 - r*i)
    beta  = 1.0 - dt*(sigma**2 * i**2 + r)
    gamma = 0.5*dt*(sigma**2 * i**2 + r*i)

    for n in range(N_t):
        tau_next = (n+1)*dt
        V_new = V.copy()
        V_new[1:-1] = alpha*V[:-2] + beta*V[1:-1] + gamma*V[2:]
        V_new[0]  = 0.0
        V_new[-1] = S_max - K*np.exp(-r * tau_next)
        V = V_new

    return float(np.interp(S0, S, V))

In [169]:
explicit_european_call(100, 100, 1, 0.07, 0.25, N_s = 100, N_t = 500)

13.33520373703625